# GDA Implementation.

Implement the Gaussian Discriminant Analysis (GDA) learning algorithm following the steps as discussed in class.

INSTRUCTION: Rename your notebook as: <br>
`firstName_LastName_Live_coding_GDA.ipynb`.

Notes: 
* Do not use any built-in functions to complete a task;
* Do not import additional libraries.

In [317]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

In [318]:
# Generate data
def generate_data():
  x, y = make_classification(n_samples= 1000, n_features=3, n_redundant=0, 
                           n_informative=3, random_state=1, 
                           n_clusters_per_class=1)
  
  return x,y

x,y= generate_data() # get data
print(x.shape, y.shape)

(1000, 3) (1000,)


In [319]:
def stackXY(x, y):
  return np.hstack((x,y))

In [ ]:
vo = np.array([[2,2,2],[1,1,1], [0,0,0]])
yo = np.array([[7],[8], [9]])
von = stackXY(vo,yo)
von

array([[2, 2, 2, 7],
       [1, 1, 1, 8],
       [0, 0, 0, 9]])

In [ ]:
n,_ = vo.shape
perm_index = np.random.permutation(n)
von = von[perm_index]
von

array([[1, 1, 1, 8],
       [2, 2, 2, 7],
       [0, 0, 0, 9]])

In [320]:
classes = len(np.unique(y))
classes

2

In [321]:
def split_data(x,y, train_size= 0.8):
    # shuffle the data to randomize the train/test split
    n,_ = x.shape
    data = stackXY(x, y)

    perm_index = np.random.permutation(n)
    data = data[perm_index]

    xtrain, ytrain = data[0:round(len(data)*train_size),0:-1], data[0:round(len(data)*train_size),-1]
    xtest, ytest = data[round(len(data)*train_size):,0:-1], data[round(len(data)*train_size):,-1]
    
    return xtrain, xtest, ytrain, ytest

In [322]:
X_train, X_test, y_train, y_test= split_data(x,y.reshape(-1,1)) # split your data into x_train, x_test, y_train, y_test
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(800, 3) (800,) (200, 3) (200,)


In [293]:
np.cov(x,rowvar=0)

array([[1.84495325, 0.02790646, 1.00137533],
       [0.02790646, 1.00170721, 0.05539176],
       [1.00137533, 0.05539176, 1.74832   ]])

In [323]:
def covariance(x, mu):
  n, d = x.shape
  sigma = np.zeros((d,d))
  # Easy way: cov= np.cov(x, rowvar=0) but do not use it. One can use it to assess his/her result.

  for i in range(d):
    #each i here has same mu
    for j in range(d):   #looking at columns

      sigma[i,j] = (1/(n- 1)) * np.sum((x.T[i] - mu[i]) @ (x.T[j] - mu[j]).T)
     
      #(x[j] - mu) @ (x[j] - mu).T
  return sigma

In [141]:
covariance(x, np.mean(x, axis=0))


array([[1.84495325, 0.02790646, 1.00137533],
       [0.02790646, 1.00170721, 0.05539176],
       [1.00137533, 0.05539176, 1.74832   ]])

In [142]:
covariance(x, mu=x.mean(0))

array([[1.84495325, 0.02790646, 1.00137533],
       [0.02790646, 1.00170721, 0.05539176],
       [1.00137533, 0.05539176, 1.74832   ]])

In [145]:
for a_class in range(3):
  ans = np.mean(1 == 1)
ans

1.0

In [324]:
class GDA:
  def __init__(self):
    ## set mu, phi and sigma to None
    self.phi  = None
    self.mu = None
    self.sigma = None
    
  def fit(self,x,y):
    k=len(np.unique(y)) # Number of class.
    d=x.shape[1]  # input dim
    m= x.shape[0] # Number of examples.
    
    ## Initialize mu, phi and sigma
    self.mu = np.zeros((k,d))
    self.sigma = np.zeros((k,d,d))
    self.phi = np.zeros((k,1))


    ## START THE LEARNING: estimate mu, phi and sigma.

    for a_class in range(k):
      
      my_idx = (y == a_class)

      #self.mu[a_class]= (np.sum(x[my_idx, :].T, axis=0)) / np.sum(my_idx) #: kxd, i.e., each row contains an individual class mu.

      #self.mu[a_class] = np.sum((my_idx @ x.T), axis = 0) / np.sum(my_idx, axis = 0)

      self.sigma[a_class] = covariance(x, self.mu[a_class])  #: kxdxd, i.e., each row contains an individual class sigma.

      self.phi[a_class] = np.mean(my_idx)     # k-dimension

    

      for col_element in range(d):

        self.mu[a_class, col_element] = np.sum((x[my_idx].T)[col_element]) / x[my_idx].shape[0]

    return self.sigma, self.mu, self.phi


  def predict_proba(self,x):
    # reshape or flatt x.
    #x= x.reshape(-1,1)
    d= x.shape[1]
    k_class= len(np.unique(y)) # Number of classes we have in our case it's k = 2
    
    ## START THE LEARNING: estimate mu, phi and sigma.
    #probability = np.zeros((x.shape[0], k_class))
    logprobability = np.zeros((x.shape[0], k_class))
    
    for i in range(k_class):

      #exp = -0.5* ( x-self.mu[0] ) @ (self.sigma[0]) @ (x - self.mu[0]).T

      xy = 1/(((2*np.pi)**(d/2)) * np.sqrt(np.linalg.det(self.sigma[i])))

      for j in range(x.shape[0]):

        #probability[j, i] = xy * np.exp((-0.5 * (x[j] -self.mu[i]) @ (self.sigma[i]) @ (x[j] - self.mu[i]).T)) * self.phi[i]
        
        logprobability[j,i] =  np.log( xy * np.exp((-0.5 * (x[j] -self.mu[i]) @ (self.sigma[i]) @ (x[j] - self.mu[i]).T)) ) + np.log( self.phi[i])

    return logprobability

  def predict(self,x):
    score = self.predict_proba(x)
    ypreds = []
    
    for i in range(x.shape[0]):
      y_pred = np.argmax(score[i])

      ypreds.append(y_pred)

    return np.array(ypreds)
  
  def accuracy(self, y, ypreds):
    return (np.mean(y == ypreds)) * 100

In [325]:
model= GDA()
model.fit(X_train,y_train)

(array([[[1.81762859e+00, 5.49151664e-04, 9.88389668e-01],
         [5.49151664e-04, 1.99393769e+00, 9.41535353e-02],
         [9.88389668e-01, 9.41535353e-02, 1.73476916e+00]],
 
        [[1.81762859e+00, 5.49151664e-04, 9.88389668e-01],
         [5.49151664e-04, 1.99393769e+00, 9.41535353e-02],
         [9.88389668e-01, 9.41535353e-02, 1.73476916e+00]]]),
 array([[ 0.97181311,  1.03390531,  1.00108928],
        [-0.9924146 ,  0.96881627, -0.90762424]]),
 array([[0.495],
        [0.505]]))

In [326]:
yproba= model.predict_proba(X_test)
yproba

array([[ -5.79964884, -24.1715064 ],
       [-47.41516987, -15.71449222],
       [-20.17727789,  -5.02392596],
       [ -8.5616509 ,  -6.08028294],
       [-18.02559027,  -4.5386666 ],
       [-14.21850626,  -4.55575777],
       [ -9.36109166, -19.33681083],
       [-20.13981503,  -5.78456406],
       [ -7.92680528, -24.06397605],
       [ -4.4906488 , -14.26116913],
       [ -9.74800158, -16.29644415],
       [ -5.11030882, -10.76364429],
       [-11.26669302, -22.7617307 ],
       [ -5.38575295,  -9.49002743],
       [ -7.03709077,  -8.22273228],
       [-24.24597775,  -6.17210087],
       [ -7.21707863, -26.78570767],
       [ -4.46808039, -17.05362883],
       [ -8.084071  ,  -8.2851529 ],
       [ -6.38870582, -17.97575692],
       [-12.24510428,  -4.74674952],
       [-16.75456338,  -5.1122326 ],
       [-11.96124963,  -4.73061026],
       [-14.60401176,  -4.7004875 ],
       [ -7.05477519, -26.89495434],
       [ -9.48761502,  -5.44936146],
       [ -4.76063463, -16.86947968],
 

In [327]:
ypreds= model.predict(X_test)
ypreds


array([0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 1])

In [328]:
model.accuracy(y_test, ypreds)

96.0